# Frankross_analytic Github

- [frankross_analytic_github link](https://github.com/kr2rohit/frankross_analytics)

In [1]:
## C42 New Order Report
import csv
import psycopg2
import pandas as pd
import numpy as np
import paramiko
from sshtunnel import SSHTunnelForwarder
import gspread
import re
from gspread_dataframe import set_with_dataframe

# EC2 instance connection details
ec2_host = '65.1.183.184'
ec2_username = 'ubuntu'
ec2_pem_key_path = 'C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem'

# AWS RDS PostgreSQL database connection details
rds_host = 'emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com'
rds_port = 5432
rds_database = 'efrprod'
rds_user = 'emamireaduser'
rds_password = 'emamireadaccess'

try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )
        cursor = connection.cursor()

        # Perform database operations here
        cursor.execute(
'''
SELECT order_Table.*
,CASE WHEN
	od IS NULL THEN 'Not First Order'
	ELSE '1st Order' END AS new_flag
,CASE
 	WHEN timeout_at IS NOT NULL THEN 'timeout'
	WHEN rejected_at IS NOT NULL THEN 'rejected'
	WHEN accepted_at IS NOT NULL THEN 'accepted'
	ELSE 'pending' end as order_action
FROM
(SELECT		
 osl.order_id		
,o.user_id as created_by_id		
,CASE		
	WHEN o.state =0 THEN 'cart'	
	WHEN o.state =1 THEN 'pre_checkout'	
	WHEN o.state =2 THEN 'Checkout'	
	WHEN o.state =6 THEN 'Delivered'	
	WHEN o.state =7 THEN 'Cancelled'	
	WHEN o.state =14  THEN 'Return_completed'	
	ELSE 'In-Progress'	
END AS status		
,o.city_id		
,o.delivery_remarks		
,o.payment_method		
,o.auto_completed		
,o.order_total_paise*1.0/100 AS order_value		
,o.shipping_total_paise*1.0/100 AS Shipping_charge		
,o.doctor_names		
--,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id		
,osl.confirmed_on		
,CASE		
	WHEN osl.channel = 0 THEN 'Call Center'	
	WHEN osl.channel = 1 THEN 'Mobile'	
	ELSE 'Website'	
END as channel_name		
,CASE		
	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'	
	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
	WHEN osl.channel = 0 THEN 'Call Center'	
	WHEN osl.channel = 1 THEN 'Saathi App'	
	ELSE 'Website'	
END as channel2		
,CASE 		
	WHEN o.state =0 THEN 'cart'	
	WHEN o.state =1 THEN 'pre_checkout'	
	WHEN o.state =2 THEN 'Checkout'	
	WHEN o.state =3 THEN 'order_received'	
	WHEN o.state =4 THEN 'shipped'	
	WHEN o.state =5 THEN 'out_for_delivery'	
	WHEN o.state =6 THEN 'Delivered'	
	WHEN o.state =7 THEN 'Cancelled'	
	WHEN o.state =14 THEN 'Return_completed'	
	WHEN o.state =17 THEN 'rescheduled'	
		ELSE 'In-Progress' END AS detailed_status
,customer_id		
,users.name AS modified_by		
,w.code AS fulfillment_center	
,Wa.code AS store_code
,DS.slot_description		
,DS.slot_date AS Expected_Delivery		
,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start		
,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end		
,co.state_changed_on + time '5:30' as cancelled_date		
,co.reason		
,co.remarks		
,CASE		
	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'	
	WHEN so.modified_by_id = 25 THEN 'Vinculum API'	
	WHEN so.modified_by_id = 44306 THEN 'Delite'	
	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'	
	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'	
	ELSE 'Others'END AS shipped_by	
,so.state_changed_on + time '5:30'AS shipped		
,ofd.state_changed_on + time '5:30'AS out_for_delivery		
,del.state_changed_on + time '5:30'AS delivered_date		
,iv.amount_paise/100*1.0 AS invoiced_amt		
,iv.invoiced_at		
,iv.wallet_amount		
,ph.number		
,us.name AS user_name		
,us.created_at AS registration_date		
,us.registration_source		
,areas.pincode		
,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order		
--,CASE WHEN 		
--	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1	
--	THEN '1st Order' ELSE 'Not First Order'		
--END AS new_flag_month	
,CASE 		
	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'		
	 WHEN del.state_changed_on + time '5:30' 		
	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'	
	 WHEN del.state_changed_on + time '5:30' 		
	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) 	
	 THEN 'Delay'	
	 ELSE 'Between Slot'		
END AS delivery_flag
,oah_t.created_at + time '5:30' as timeout_at
,oah_a.created_at + time '5:30' as accepted_at
,oah_r.created_at + time '5:30' as rejected_at

FROM		
	(SELECT	
	order_id	
	,MAX(modified_by_id) modified_by_id	
	,MAX(channel) channel	
	,MAX(state_changed_on + time '5:30') AS confirmed_on	
	FROM order_status_logs	
	WHERE state_changed_on >'2024-12-31 18:30' AND to_state = 3	
	GROUP BY order_id) AS osl	
LEFT JOIN orders o ON o.id = osl.order_id		
LEFT JOIN users ON users.id = osl.modified_by_id		
LEFT JOIN warehouses w ON w.id = o.fulfillment_center_id
LEFT JOIN warehouses wa ON wa.id = o.store_id
LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id		
LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders		
LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders		
LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery		
LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery		
LEFT JOIN invoices iv ON iv.order_id = osl.order_id		
LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL		
LEFT JOIN users us ON us.id = o.user_id 		
LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id 		
LEFT JOIN areas ON areas.id = addresses.area_id
LEFT JOIN order_assignment_histories oah_t on oah_t.order_id = osl.order_id AND oah_t.action = 'time_out'
LEFT JOIN order_assignment_histories oah_a on oah_a.order_id = osl.order_id AND oah_a.action = 'accept'
LEFT JOIN order_assignment_histories oah_r on oah_r.order_id = osl.order_id AND oah_r.action = 'reject'
ORDER BY osl.confirmed_on DESC) as order_Table
LEFT JOIN 
(SELECT od FROM
(SELECT ord.*,
ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number 
FROM
(
SELECT 
order_id as od
,MAX(u.created_at) as registered_at
,MAX(o.user_id) as user_id
,MAX(state_changed_on + time '5:30') as confirmed_on
,MAX(channel) as channel
,MAX(modified_by_id) as modified_by
FROM order_status_logs osl 
LEFT JOIN orders o on o.id = osl.order_id
LEFT JOIN users u on u.id = o.user_id
WHERE to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
GROUP BY od
) AS ord) 
AS ordr
WHERE order_number = 1
AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
--AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
AND confirmed_on >= '2024-12-31 18:30:00'
--AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
) as first_order
on first_order.od = order_Table.order_id
order by order_Table.confirmed_on DESC

''')
        #records = cursor.fetchall()
#
        ## Write records to a CSV file
        #with open('c42_test.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow([desc[0] for desc in cursor.description])  # Write column headers
        #    writer.writerows(records)
        records = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
        df = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
        #df.to_csv('c42.csv')
        df = df[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
         'payment_method', 'new_flag', 'order_value', 'shipping_charge',
         'detailed_status', 'out_for_delivery', 'confirmed_on', 'channel_name',
         'shipped', 'fulfillment_center', 'slot_description', 'expected_delivery',
         'modified_by', 'reason', 'remarks', 'cancelled_date',
         'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
         'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number', 'store_code','pincode','wallet_amount'
         ,'order_action']]
        print("Data fetched and saved c42 to df")

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)

finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

C:\Users\kumar.rohit.KUMARROHIT-LPT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Data fetched and saved c42 to df
PostgreSQL connection is closed


In [3]:
df.columns

Index(['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
       'payment_method', 'new_flag', 'order_value', 'shipping_charge',
       'detailed_status', 'out_for_delivery', 'confirmed_on', 'channel_name',
       'shipped', 'fulfillment_center', 'slot_description',
       'expected_delivery', 'modified_by', 'reason', 'remarks',
       'cancelled_date', 'delivered_date', 'channel2', 'invoiced_amt',
       'shipped_by', 'exp_delivery_start', 'exp_delivery_end', 'delivery_flag',
       'number', 'store_code', 'pincode', 'wallet_amount', 'order_action'],
      dtype='object')

In [ ]:
# To Extrach First order after '2022-12-31 18:30:00'

SELECT *
FROM (
    SELECT ord.*,
           ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number 
    FROM (
        SELECT 
            order_id AS od,
            MAX(u.created_at) AS registered_at,
            MAX(o.user_id) AS user_id,
            MAX(state_changed_on + time '5:30') AS confirmed_on,
            MAX(channel) AS channel,
            MAX(modified_by_id) AS modified_by
        FROM order_status_logs osl 
        LEFT JOIN orders o ON o.id = osl.order_id
        LEFT JOIN users u ON u.id = o.user_id
        WHERE to_state = 3 
          AND state_changed_on > '2022-12-31 18:30:00'
        GROUP BY order_id
    ) AS ord
) AS ordr
WHERE order_number = 1
  AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
  AND confirmed_on >= '2024-11-30 18:30:00'
  --AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE);



In [ ]:
#User Segmentation 2024

WITH user_monthly_orders AS (
    SELECT
        o.user_id,
        DATE_TRUNC('month', osl.state_changed_on) AS order_month,
        COUNT(o.id) AS monthly_order_count,
        SUM(o.order_total_paise / 100.0) AS monthly_order_value, -- Total value per month
        AVG(o.order_total_paise / 100.0) AS avg_ticket_size -- Average ticket size
    FROM 
        order_status_logs osl
    LEFT JOIN 
        orders o ON o.id = osl.order_id
    WHERE 
        osl.to_state = 3 
        AND osl.state_changed_on > '2023-12-31 18:31:00'
		--AND channel in (1,4,0)
    GROUP BY 
        o.user_id, DATE_TRUNC('month', osl.state_changed_on)
),
user_summary AS (
    SELECT
        user_id,
        AVG(monthly_order_count) AS avg_monthly_orders,
        SUM(monthly_order_count) AS total_orders,
        SUM(monthly_order_value) AS customer_yearly_value, -- Total order value for the year
        COUNT(DISTINCT order_month) AS active_months,
        AVG(avg_ticket_size) AS avg_ticket_size
    FROM 
        user_monthly_orders
    GROUP BY 
        user_id
)
SELECT 
    user_id,
    avg_monthly_orders,
    total_orders,
    active_months,
    avg_ticket_size,
    customer_yearly_value,
    CASE
        WHEN active_months >= 10 AND avg_monthly_orders >= 1 AND avg_ticket_size > 500 THEN 'Platinum'
        WHEN active_months >= 7 AND avg_monthly_orders >= 1 AND avg_ticket_size > 500 THEN 'Diamond'
        WHEN active_months >= 5 AND avg_monthly_orders >= 1 AND avg_ticket_size > 500 THEN 'Gold'
        WHEN active_months >= 3 AND avg_monthly_orders >=1 AND avg_ticket_size > 500 
		THEN 'Silver'
        ELSE 'Others'
    END AS user_category
FROM 
    user_summary;



In [ ]:
#New Order Reoprt with correct 1st order 

SELECT order_Table.*
,CASE WHEN
	od IS NULL THEN 'Not First Order'
	ELSE '1st Order' END AS new_flag
FROM
(SELECT		
 osl.order_id		
,o.user_id as created_by_id		
,CASE		
	WHEN o.state =0 THEN 'cart'	
	WHEN o.state =1 THEN 'pre_checkout'	
	WHEN o.state =2 THEN 'Checkout'	
	WHEN o.state =6 THEN 'Delivered'	
	WHEN o.state =7 THEN 'Cancelled'	
	WHEN o.state =14  THEN 'Return_completed'	
	ELSE 'In-Progress'	
END AS status		
,o.city_id		
,o.delivery_remarks		
,o.payment_method		
,o.auto_completed		
,o.order_total_paise*1.0/100 AS order_value		
,o.shipping_total_paise*1.0/100 AS Shipping_charge		
,o.doctor_names		
--,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id		
,osl.confirmed_on		
,CASE		
	WHEN osl.channel = 0 THEN 'Call Center'	
	WHEN osl.channel = 1 THEN 'Mobile'	
	ELSE 'Website'	
END as channel_name		
,CASE		
	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'	
	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599) then 'Ecom_CC'
	WHEN osl.channel = 0 THEN 'Call Center'	
	WHEN osl.channel = 1 THEN 'Saathi App'	
	ELSE 'Website'	
END as channel2		
,CASE 		
	WHEN o.state =0 THEN 'cart'	
	WHEN o.state =1 THEN 'pre_checkout'	
	WHEN o.state =2 THEN 'Checkout'	
	WHEN o.state =3 THEN 'order_received'	
	WHEN o.state =4 THEN 'shipped'	
	WHEN o.state =5 THEN 'out_for_delivery'	
	WHEN o.state =6 THEN 'Delivered'	
	WHEN o.state =7 THEN 'Cancelled'	
	WHEN o.state =14 THEN 'Return_completed'	
	WHEN o.state =17 THEN 'rescheduled'	
		ELSE 'In-Progress' END AS detailed_status
,customer_id		
,users.name AS modified_by		
,w.code AS dc_code		
,DS.slot_description		
,DS.slot_date AS Expected_Delivery		
,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start		
,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end		
,co.state_changed_on + time '5:30' as cancelled_date		
,co.reason		
,co.remarks		
,CASE		
	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'	
	WHEN so.modified_by_id = 25 THEN 'Vinculum API'	
	WHEN so.modified_by_id = 44306 THEN 'Delite'	
	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'	
	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'	
	ELSE 'Others'END AS shipped_by	
,so.state_changed_on + time '5:30'AS shipped		
,ofd.state_changed_on + time '5:30'AS out_for_delivery		
,del.state_changed_on + time '5:30'AS delivered_date		
,iv.amount_paise/100*1.0 AS invoiced_amt		
,iv.invoiced_at		
,iv.wallet_amount		
,ph.number		
,us.name AS user_name		
,us.created_at AS registration_date		
,us.registration_source		
,areas.pincode		
,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order		
--,CASE WHEN 		
--	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1	
--	THEN '1st Order' ELSE 'Not First Order'		
--END AS new_flag_month	
,CASE 		
	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'		
	 WHEN del.state_changed_on + time '5:30' 		
	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'	
	 WHEN del.state_changed_on + time '5:30' 		
	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) 	
	 THEN 'Delay'	
	 ELSE 'Between Slot'		
END AS delivery_flag		
FROM		
	(SELECT	
	order_id	
	,MAX(modified_by_id) modified_by_id	
	,MAX(channel) channel	
	,MAX(state_changed_on + time '5:30') AS confirmed_on	
	FROM order_status_logs	
	WHERE state_changed_on >'2024-11-30 18:30' AND to_state = 3	
	GROUP BY order_id) AS osl	
LEFT JOIN orders o ON o.id = osl.order_id		
LEFT JOIN users ON users.id = osl.modified_by_id		
LEFT JOIN warehouses w ON w.id = o.store_id		
LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id		
LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders		
LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders		
LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery		
LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery		
LEFT JOIN invoices iv ON iv.order_id = osl.order_id		
LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL		
LEFT JOIN users us ON us.id = o.user_id 		
LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id 		
LEFT JOIN areas ON areas.id = addresses.area_id		
ORDER BY osl.confirmed_on DESC) as order_Table
LEFT JOIN 
(SELECT od FROM
(SELECT ord.*,
ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number 
FROM
(
SELECT 
order_id as od
,MAX(u.created_at) as registered_at
,MAX(o.user_id) as user_id
,MAX(state_changed_on + time '5:30') as confirmed_on
,MAX(channel) as channel
,MAX(modified_by_id) as modified_by
FROM order_status_logs osl 
LEFT JOIN orders o on o.id = osl.order_id
LEFT JOIN users u on u.id = o.user_id
WHERE to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
GROUP BY od
) AS ord) 
AS ordr
WHERE order_number = 1
AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
--AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
AND confirmed_on >= CURRENT_DATE - INTERVAL '2 MONTH'
AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
) as first_order
on first_order.od = order_Table.order_id

In [ ]:
old order with wrong 1st order flag

 
SELECT		
 osl.order_id		
,o.user_id as created_by_id		
,CASE		
	WHEN o.state =0 THEN 'cart'	
	WHEN o.state =1 THEN 'pre_checkout'	
	WHEN o.state =2 THEN 'Checkout'	
	WHEN o.state =6 THEN 'Delivered'	
	WHEN o.state =7 THEN 'Cancelled'	
	WHEN o.state =14  THEN 'Return_completed'	
	ELSE 'In-Progress'	
END AS status		
,o.city_id		
,o.delivery_remarks		
,o.payment_method		
,o.auto_completed		
,o.order_total_paise*1.0/100 AS order_value		
,o.shipping_total_paise*1.0/100 AS Shipping_charge		
,o.doctor_names		
--,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id		
,osl.confirmed_on		
,CASE		
	WHEN osl.channel = 0 THEN 'Call Center'	
	WHEN osl.channel = 1 THEN 'Mobile'	
	ELSE 'Website'	
END as channel_name		
,CASE		
	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'	
	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599) then 'Ecom_CC'
	WHEN osl.channel = 0 THEN 'Call Center'	
	WHEN osl.channel = 1 THEN 'Saathi App'	
	ELSE 'Website'	
END as channel2		
,CASE 		
	WHEN o.state =0 THEN 'cart'	
	WHEN o.state =1 THEN 'pre_checkout'	
	WHEN o.state =2 THEN 'Checkout'	
	WHEN o.state =3 THEN 'order_received'	
	WHEN o.state =4 THEN 'shipped'	
	WHEN o.state =5 THEN 'out_for_delivery'	
	WHEN o.state =6 THEN 'Delivered'	
	WHEN o.state =7 THEN 'Cancelled'	
	WHEN o.state =14 THEN 'Return_completed'	
	WHEN o.state =17 THEN 'rescheduled'	
		ELSE 'In-Progress' END AS detailed_status
,customer_id		
,users.name AS modified_by		
,w.code AS dc_code		
,DS.slot_description		
,DS.slot_date AS Expected_Delivery		
,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start		
,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end		
,co.state_changed_on + time '5:30' as cancelled_date		
,co.reason		
,co.remarks		
,CASE		
	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'	
	WHEN so.modified_by_id = 25 THEN 'Vinculum API'	
	WHEN so.modified_by_id = 44306 THEN 'Delite'	
	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'	
	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'	
	ELSE 'Others'END AS shipped_by	
,so.state_changed_on + time '5:30'AS shipped		
,ofd.state_changed_on + time '5:30'AS out_for_delivery		
,del.state_changed_on + time '5:30'AS delivered_date		
,iv.amount_paise/100*1.0 AS invoiced_amt		
,iv.invoiced_at		
,iv.wallet_amount		
,ph.number		
,us.name AS user_name		
,us.created_at AS registration_date		
,us.registration_source		
,areas.pincode		
,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order		
,CASE WHEN 		
	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1	
	THEN '1st Order' ELSE 'Not First Order'		
END AS new_flag		
,CASE 		
	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'		
	 WHEN del.state_changed_on + time '5:30' 		
	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'	
	 WHEN del.state_changed_on + time '5:30' 		
	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) 	
	 THEN 'Delay'	
	 ELSE 'Between Slot'		
END AS delivery_flag		
FROM		
	(SELECT	
	order_id	
	,MAX(modified_by_id) modified_by_id	
	,MAX(channel) channel	
	,MAX(state_changed_on + time '5:30') AS confirmed_on	
	FROM order_status_logs	
	WHERE state_changed_on >'2024-11-30 18:30' AND to_state = 3	
	GROUP BY order_id) AS osl	
LEFT JOIN orders o ON o.id = osl.order_id		
LEFT JOIN users ON users.id = osl.modified_by_id		
LEFT JOIN warehouses w ON w.id = o.store_id		
LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id		
LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders		
LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders		
LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery		
LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery		
LEFT JOIN invoices iv ON iv.order_id = osl.order_id		
LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL		
LEFT JOIN users us ON us.id = o.user_id 		
LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id 		
LEFT JOIN areas ON areas.id = addresses.area_id		
ORDER BY osl.confirmed_on DESC		

In [ ]:
1st order in last two months

With ordr as (select ord.*,
row_number() over (partition by ord.user_id order by ord.confirmed_on) as order_number 
FROM
(
SELECT 
order_id
,Max(u.created_at) as registered_at
,Max(o.user_id) as user_id
,MAX(state_changed_on + time '5:30') as confirmed_on
,max(channel) as channel
,max(modified_by_id) as modified_by
FROM order_status_logs osl 
left join orders o on o.id = osl.order_id
left join users u on u.id = o.user_id
where to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
group by order_id
) as ord)
SELECT * FROM ordr where order_number = 1
AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '2 MONTH'



In [ ]:
# Purchaser List 2024

SELECT 
o.user_id
,Max(u.name)  cust_name
,Max(ph.number) cust_number
FROM order_status_logs osl
LEFT JOIN orders o on o.id = osl.order_id
LEFT JOIN users u on u.id = o.user_id
LEFT JOIN phones ph on ph.user_id = o.user_id and deleted_at is null
where state_changed_on > '2023-12-31 18:30:00' and to_state = 3
group by o.user_id 

In [1]:
#churn Customer List - All Channel

SELECT churn_user.*, users.name,users.email, phones.number FROM (SELECT DISTINCT o.user_id  
FROM order_status_logs osl
LEFT JOIN orders o ON o.id = osl.order_id
WHERE to_state = 3 
  AND state_changed_on < '2023-12-31 18:30'  ---ordered in this period
  AND channel IN (1,4,0)
EXCEPT
SELECT DISTINCT o.user_id
FROM order_status_logs osl
LEFT JOIN orders o ON o.id = osl.order_id
WHERE to_state = 3 
  AND state_changed_on > '2023-12-31 18:30' --- but did not ordered in this period
  AND channel IN (1,4,0)) churn_user
  
LEFT JOIN users ON users.id = churn_user.user_id
LEFT JOIN phones ON phones.user_id = churn_user.user_id AND deleted_at IS NULL

In [ ]:
Sales With User Details

SELECT
    line_items.order_id as ord_id,
	o.user_id,
	phones.number,
	users.name,
    products.name as product_name,	
    CASE 
        WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
        WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
        WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599) then 'Ecom_CC'
        WHEN osl.channel = 0 THEN 'Call Center'
        WHEN osl.channel = 1 THEN 'Saathi_App'
        ELSE 'Website'
    END as channel,
    osl.state_changed_on + time '5:30' as confirmed_on,
	ip.property_value as d_profile,
	ip2.property_value as actute_chronic
FROM 
    line_items 
    LEFT JOIN variants ON line_items.variant_id = variants.id	
    LEFT JOIN products ON variants.product_id = products.id	
    LEFT JOIN product_types ON products.product_type_id = product_types.id	
    LEFT JOIN categories ON categories.id = product_types.category_id
    LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
    LEFT JOIN orders o ON o.id = osl.order_id
    LEFT JOIN distribution_centers dc ON dc.id = o.store_id
	LEFT JOIN item_properties ip ON ip.item_id = line_items.variant_id AND ip.property_id IN ('1351')
	LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
	LEFT JOIN phones on phones.user_id = o.user_id and deleted_at is null
	LEFT JOIN users on users.id = o.user_id 
WHERE 
    line_items.order_id IN ( 
        SELECT order_id 
        FROM order_status_logs as ols
        LEFT JOIN orders ON orders.id = ols.order_id
        WHERE 
            state_changed_on >= '2023-12-31 18:30:00.000000' 
            AND to_state = 3
            AND city_id = 13
    )
    AND osl.to_state = 3
	AND ip.property_value in ('CNS disorder','Cardiovascular disease','Diabetic','Hypertension')

In [ ]:
# New Bounced Orders

SELECT * FROM 
(with cte_bounce_orders as (
SELECT 
    o.order_id,
    orders.state,
    v.sku,
    v.name,
    o.deficit_quantity,
    w.store_name AS Level_1_store,
    ar.pincode,
    ols.state_changed_on + INTERVAL '5 hours 30 minutes' AS confirmed_on,
	CONCAT (o.order_id,v.sku) as order_fr_code,
    CASE
	WHEN orders.customer_id IS NULL AND ols.channel = 1 THEN 'Mobile'
	WHEN ols.channel = 0 AND COALESCE(orders.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
	WHEN ols.channel = 0 THEN 'Call Center'
	WHEN ols.channel = 1 THEN 'Saathi App'
	ELSE 'Website'
END as channel
FROM 
    order_deficit_quantity_logs o
LEFT JOIN 
    warehouses w ON o.warehouse_id = w.id
LEFT JOIN 
    areas ar ON o.area_id = ar.id
LEFT JOIN 
    variants v ON v.id = o.variant_id
LEFT JOIN 
    orders ON orders.id = o.order_id
LEFT JOIN 
    order_status_logs ols ON ols.order_id = o.order_id
WHERE
    ols.to_state = 3
    AND ols.state_changed_on > '2024-11-30 18:30:00'
    AND orders.state > 2
    AND ols.state_changed_on + INTERVAL '5 hours 30 minutes' > '2024-11-30 18:30:00'
)
select * From cte_bounce_orders where confirmed_on > '2024-11-30 18:30:00') Bounce_tbl

left join 

(
SELECT 
CONCAT (order_id,'FR-',variant_id) as order_fr_code,
quantity as order_qty
FROM line_items where order_id in 
(
SELECT
order_id
--,MAX(modified_by_id) modified_by_id
--,MAX(channel) channel
--,MAX(state_changed_on + time '5:30') AS confirmed_on
FROM order_status_logs
WHERE state_changed_on >'2024-11-30 18:30' AND to_state = 3
GROUP BY order_id
)) ord_qty
on ord_qty.order_fr_code = Bounce_tbl.order_fr_code

In [ ]:
Welcome credit  50 rs  to registerd users

SELECT
hw.user_id
,users.registration_source
,hw.id
,hw.earned_points
,hw.spent_points
,hw.available_points
,hwt.created_at
,hwt.transaction_type
,hwt.points
,hwt.summary
FROM public.health_wallet_transactions hwt 
left join health_wallets hw on hw.id = hwt.health_wallet_id
left join users on users.id = hw.user_id
where summary like '%Welcome Credit%'
AND hwt.created_at > '2024-11-30 18:30:00' AND hwt.created_at < '2024-12-19 18:30:00'



In [ ]:
Active Store Panel

SELECT 
store_managers.user_id
,w.code
,store_managers.user_id
,users.name
,users.email
FROM store_managers 
left join
warehouses w on w.id = store_managers.warehouse_id
left join users on users.id = store_managers.user_id

In [ ]:
# Reg But Not ordered User phone number

Select user_id,phones.number From phones where user_id in
(SELECT id FROm users where created_at > '2024-10-31 18:31:00'
 created_at < '2024-11-01 18:31:00'
 and registration_source in (1,4)
and id not in 
(select modified_by_id from order_status_logs where to_state = 3 and state_changed_on > '2024-10-31 18:31:00'))

In [1]:
#Acute Chronic List


import csv
import psycopg2
import pandas as pd
import numpy as np
import paramiko
from sshtunnel import SSHTunnelForwarder
# EC2 instance connection details
ec2_host = '65.1.183.184'
ec2_username = 'ubuntu'
ec2_pem_key_path = "C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem"
# AWS RDS PostgreSQL database connection details
rds_host = 'emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com'
rds_port = 5432
rds_database = 'efrprod'
rds_user = 'emamireaduser'
rds_password = 'emamireadaccess'
try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )

        # Chronic Data Query
        chronic_query = '''
        SELECT DISTINCT
            o.user_id,
            ip2.property_value AS actute_chronic
        FROM 
            line_items 
            LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
            LEFT JOIN orders o ON o.id = osl.order_id
            LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
        WHERE 
            line_items.order_id IN ( 
                SELECT order_id 
                FROM order_status_logs AS ols
                LEFT JOIN orders ON orders.id = ols.order_id
                WHERE 
                    state_changed_on >= '2015-12-31 18:30:00.000000' 
                    AND to_state = 3
                    AND city_id = 13
            )
            AND osl.to_state = 3 AND ip2.property_value = 'Chronic'
        '''

        # Acute Data Query
        acute_query = '''
         SELECT DISTINCT
            o.user_id,
            ip2.property_value AS actute_chronic
        FROM 
            line_items 
            LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
            LEFT JOIN orders o ON o.id = osl.order_id
            LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
        WHERE 
            line_items.order_id IN ( 
                SELECT order_id 
                FROM order_status_logs AS ols
                LEFT JOIN orders ON orders.id = ols.order_id
                WHERE 
                    state_changed_on >= '2015-12-31 18:30:00.000000' 
                    AND to_state = 3
                    AND city_id = 13
            )
            AND osl.to_state = 3 AND ip2.property_value = 'Acute'
        '''

        # Fetch DataFrames
        chronic_df = pd.read_sql(chronic_query, connection)
        acute_df = pd.read_sql(acute_query, connection)
        
        # Identify users exclusive to the acute dataset
        acute_only_df = acute_df[~acute_df['user_id'].isin(chronic_df['user_id'])]

        # Combine acute-only data with the full chronic data
        combined_df = pd.concat([acute_only_df, chronic_df], ignore_index=True)

        # Save to Excel with one sheet
        with pd.ExcelWriter('acute_chronic_master.xlsx') as writer:
            combined_df.to_excel(writer, sheet_name='acute_chronic_master', index=False)

        print("Data saved to acute_chronic_master.xlsx")

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    if connection:
        connection.close()
        print("PostgreSQL connection is closed")


C:\Users\kumar.rohit.KUMARROHIT-LPT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Data saved to acute_chronic_master.xlsx
PostgreSQL connection is closed


## C42 New Order Report

In [2]:
## C42 New Order Report
import csv
import psycopg2
import pandas as pd
import numpy as np
import paramiko
from sshtunnel import SSHTunnelForwarder
import gspread
import re
from gspread_dataframe import set_with_dataframe

# EC2 instance connection details
ec2_host = '65.1.183.184'
ec2_username = 'ubuntu'
ec2_pem_key_path = 'C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem'

# AWS RDS PostgreSQL database connection details
rds_host = 'emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com'
rds_port = 5432
rds_database = 'efrprod'
rds_user = 'emamireaduser'
rds_password = 'emamireadaccess'

try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )
        cursor = connection.cursor()

        # Perform database operations here
        cursor.execute(
'''
 
SELECT
osl.order_id
,o.user_id as created_by_id
,CASE
	WHEN o.state =0 THEN 'cart'
	WHEN o.state =1 THEN 'pre_checkout'
	WHEN o.state =2 THEN 'Checkout'
	WHEN o.state =6 THEN 'Delivered'
	WHEN o.state =7 THEN 'Cancelled'
	WHEN o.state =14  THEN 'Return_completed'
	ELSE 'In-Progress'
END AS status
,o.city_id
,o.delivery_remarks
,o.payment_method
,o.auto_completed
,o.order_total_paise*1.0/100 AS order_value
,o.shipping_total_paise*1.0/100 AS Shipping_charge
,o.doctor_names
--,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id
,osl.confirmed_on
,CASE
	WHEN osl.channel = 0 THEN 'Call Center'
	WHEN osl.channel = 1 THEN 'Mobile'
	ELSE 'Website'
END as channel_name
,CASE
	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
	WHEN osl.channel = 0 THEN 'Call Center'
	WHEN osl.channel = 1 THEN 'Saathi App'
	ELSE 'Website'
END as channel2
,CASE 
	WHEN o.state =0 THEN 'cart'
	WHEN o.state =1 THEN 'pre_checkout'
	WHEN o.state =2 THEN 'Checkout'
	WHEN o.state =3 THEN 'order_received'
	WHEN o.state =4 THEN 'shipped'
	WHEN o.state =5 THEN 'out_for_delivery'
	WHEN o.state =6 THEN 'Delivered'
	WHEN o.state =7 THEN 'Cancelled'
	WHEN o.state =14 THEN 'Return_completed'
	WHEN o.state =17 THEN 'rescheduled'
		ELSE 'In-Progress' END AS detailed_status
,customer_id
,users.name AS modified_by
,w.code AS dc_code
,DS.slot_description
,DS.slot_date AS Expected_Delivery
,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start
,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end
,co.state_changed_on + time '5:30' as cancelled_date
,co.reason
,co.remarks
,CASE
	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'
 	WHEN so.modified_by_id = 25 THEN 'Vinculum API'
 	WHEN so.modified_by_id = 44306 THEN 'Delite'
 	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'
 	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'
 	ELSE 'Others'END AS shipped_by
,so.state_changed_on + time '5:30'AS shipped
,ofd.state_changed_on + time '5:30'AS out_for_delivery
,del.state_changed_on + time '5:30'AS delivered_date
,iv.amount_paise/100*1.0 AS invoiced_amt
,iv.invoiced_at
,iv.wallet_amount
,ph.number
,us.name AS user_name
,us.created_at AS registration_date
,us.registration_source
,areas.pincode
,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order
,CASE WHEN 
	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1
    THEN '1st Order' ELSE 'Not First Order'
END AS new_flag
,CASE 
     WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'
     WHEN del.state_changed_on + time '5:30' 
	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'
     WHEN del.state_changed_on + time '5:30' 
	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) 
	 THEN 'Delay'
     ELSE 'Between Slot'
END AS delivery_flag
FROM
	(SELECT
	order_id
	,MAX(modified_by_id) modified_by_id
	,MAX(channel) channel
	,MAX(state_changed_on + time '5:30') AS confirmed_on
	FROM order_status_logs
	WHERE state_changed_on >'2024-10-31 18:30' AND to_state = 3
	GROUP BY order_id) AS osl
LEFT JOIN orders o ON o.id = osl.order_id
LEFT JOIN users ON users.id = osl.modified_by_id
LEFT JOIN warehouses w ON w.id = o.store_id
LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id
LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders
LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders
LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery
LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery
LEFT JOIN invoices iv ON iv.order_id = osl.order_id
LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL
LEFT JOIN users us ON us.id = o.user_id 
LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id 
LEFT JOIN areas ON areas.id = addresses.area_id
ORDER BY osl.confirmed_on DESC

 
''')
        #records = cursor.fetchall()
#
        ## Write records to a CSV file
        #with open('c42_test.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow([desc[0] for desc in cursor.description])  # Write column headers
        #    writer.writerows(records)
        records = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
        df = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
        df = df[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
         'payment_method', 'new_flag', 'order_value', 'shipping_charge',
         'detailed_status', 'out_for_delivery', 'confirmed_on', 'channel_name',
         'shipped', 'dc_code', 'slot_description', 'expected_delivery',
         'modified_by', 'reason', 'remarks', 'cancelled_date',
         'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
         'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number', 'user_name', 'pincode']]
        print("Data fetched and saved to c42.csv and df")

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)

finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

# Updating Data into Google sheets
gsheet_name = 'Summary_Epharmacy' #This google sheet will be updated
tab_name = 'C42_Combined' # This particular tab is to be updated

def write_df_to_gsheet (gsheet_name,tab_name,df):#Updating In Summary_Epharmacy
    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
    sh = gc.open_by_key("1etqrto99N3Tmv9Z-svwa4OtFjUewpP0KwMiflq36hrg") #Key Of the google sheet - Summary_Epharmacy
    worksheet = sh.worksheet(tab_name)
    set_with_dataframe(worksheet,df)
write_df_to_gsheet(gsheet_name,tab_name,df)
df.to_csv('c42_f.csv')
print("Updated_at", pd.Timestamp.now())

Data fetched and saved to c42.csv and df
PostgreSQL connection is closed
Updated_at 2024-12-09 14:17:25.607077


In [ ]:
Check Stock sync Request for store 020 and article P32693

SELECT *
FROM public.ecogreen_sync_requests
WHERE Parameters IS NOT NULL
  AND Parameters like ('%c_item_code: P32693%')
  AND Parameters like ('%032020%') -- ## 032 is added for all store as prefix


In [1]:
# Monthly Sales
import pandas as pd
import csv
import gspread
import psycopg2
import paramiko
from sshtunnel import SSHTunnelForwarder

# EC2 instance connection details
ec2_host = '65.1.183.184'
ec2_username = 'ubuntu'
ec2_pem_key_path = 'C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem'

# AWS RDS PostgreSQL database connection details
rds_host = 'emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com'
rds_port = 5432
rds_database = 'efrprod'
rds_user = 'emamireaduser'
rds_password = 'emamireadaccess'

try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )
        cursor = connection.cursor()

        # Perform database operations here
        cursor.execute('''
    SELECT
    line_items.order_id as ord_id,	
    line_items.variant_id,
    products.name as product_name,	
    categories.name as category,
    line_items.quantity,	
    line_items.mrp_paise * 1.0 / 100 as MRP,	
    line_items.sales_price_paise * 1.0 / 100 as Sales_price,	
    line_items.total_paise * 1.0 / 100 as Total_Price,	
    line_items.discount_amount_paise * 1.0 / 100 as Discount,
    CASE 
        WHEN osl.channel = 0 THEN 'Call Center'
        WHEN osl.channel = 1 THEN 'Mobile'
        ELSE 'Website' 
    END as channel_1,
    CASE 
        WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
        WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
        WHEN osl.channel = 0 THEN 'Call Center'
        WHEN osl.channel = 1 THEN 'Saathi_App'
        ELSE 'Website'
    END as channel,
    osl.state_changed_on + interval '5:30' hour as confirmed_on,
    CASE 
        WHEN o.state = 0 THEN 'cart'
        WHEN o.state = 1 THEN 'pre_checkout'
        WHEN o.state = 2 THEN 'Checkout'
        WHEN o.state = 6 THEN 'Delivered'
        WHEN o.state = 7 THEN 'Cancelled'
        WHEN o.state = 14 THEN 'Return_completed'
        ELSE 'In-Progress' 
    END as status,
    dc.dc_code,
	ip.property_value as d_profile,
	ip2.property_value as actute_chronic
FROM 
    line_items 
    LEFT JOIN variants ON line_items.variant_id = variants.id	
    LEFT JOIN products ON variants.product_id = products.id	
    LEFT JOIN product_types ON products.product_type_id = product_types.id	
    LEFT JOIN categories ON categories.id = product_types.category_id
    LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
    LEFT JOIN orders o ON o.id = osl.order_id
    LEFT JOIN distribution_centers dc ON dc.id = o.store_id
	LEFT JOIN item_properties ip ON ip.item_id = line_items.variant_id AND ip.property_id IN ('1351')
	LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
WHERE 
    line_items.order_id IN ( 
        SELECT order_id 
        FROM order_status_logs as ols
        LEFT JOIN orders ON orders.id = ols.order_id
        WHERE 
            state_changed_on >= '2024-03-31 18:30:00.000000' 
            AND to_state = 3
            AND city_id = 13
    )
    AND osl.to_state = 3''')
        records = cursor.fetchall()

        # Write record to a CSV file
        with open('monthly_sales.csv', 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([desc[0] for desc in cursor.description])  # Write column headers
            writer.writerows(records)

        print("Data fetched and saved to monthly_sales.csv")

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)

finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
print("Updated at", pd.Timestamp.now())

C:\Users\kumar.rohit.KUMARROHIT-LPT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Data fetched and saved to monthly_sales.csv
PostgreSQL connection is closed
Updated at 2024-10-21 13:49:47.718526


In [ ]:
2nd Order Cashback

select user_id from (
select user_id, count(uid) as count_orders from (
select user_id, orders.id as uid from orders, order_status_logs where user_id in (
select orders.user_id from order_promotions, order_status_logs, orders
where promotion_id in (2179,2037,2055,2066,2078,2085,2092,2129,2137,2169,2170)
and order_status_logs.to_state = 6
and order_status_logs.order_id = order_promotions.order_id
and order_status_logs.order_id = orders.id)
and order_status_logs.order_id = orders.id
and order_status_logs.to_state = 6) as foo
group by user_id
having count(uid) >= 2) as boo
where boo.user_id not in (
select distinct(user_id) from health_wallets, health_wallet_transactions 
where health_wallet_id = health_wallets.id
--and summary like any ('%2nd order cashback%', '%2nd Order Cashback%', '%2nd order Cashback%', '%2nd Order cashback%',
--					'%2nd Order Cashback%')
--and summary = all ('{%2nd order cashback%, %2nd order Cashback%, 
--					  %2nd Order cashback%, %2nd Order Cashback%}')
and (summary like '%2nd order cashback%'
	 or summary like '%2nd order Cashback%' 
	 or summary like '%2nd Order cashback%'  
	 or summary like '%2nd Order Cashback%'))

In [ ]:
Stock Sync report

SELECT warehouses.eg_code
,max(warehouses.code) as DC_code
,max(warehouses.store_name) as warehouse
,max(last_synced_at) as last_synced_date 
FROM public.warehouse_variants
left join warehouses on warehouses.id= warehouse_variants.warehouse_id
group by warehouses.eg_code


-- Stores synced today
SELECT 
    warehouses.eg_code,
    MAX(warehouses.code) AS DC_code,
    MAX(warehouses.store_name) AS warehouse,
    MAX(warehouse_variants.last_synced_at) AS last_synced_date
FROM public.warehouse_variants
LEFT JOIN warehouses ON warehouses.id = warehouse_variants.warehouse_id
WHERE warehouse_variants.last_synced_at >= current_date
GROUP BY warehouses.eg_code;

----#Stock did not Synced Today
WITH synced_stores AS (
    SELECT 
        warehouses.eg_code,
        MAX(warehouse_variants.warehouse_id) AS warehouse_id_c42,
        MAX(warehouses.store_name) AS warehouse,
        MAX(warehouse_variants.last_synced_at) AS last_synced_date
    FROM public.warehouse_variants
    LEFT JOIN warehouses 
        ON warehouses.id = warehouse_variants.warehouse_id
    WHERE warehouse_variants.last_synced_at >= current_date
    GROUP BY warehouses.eg_code
)

-- Fetching stores that did not sync today
SELECT 
all_stores.*,
synced_stores.eg_code,
synced_stores.warehouse,
synced_stores.last_synced_date
FROM (
    SELECT unnest(ARRAY[277, 256, 232, 271, 296, 219, 306, 203, 267, 266, 288, 318, 262, 260, 243, 254, 
                        286, 270, 235, 236, 350, 257, 237, 225, 202, 250, 300, 221, 226, 279, 357, 
                        377, 391, 430,426, 427, 443, 446, 457, 471, 496, 505]) AS warehouse_id_c42
) AS all_stores
LEFT JOIN synced_stores ON all_stores.warehouse_id_c42 = synced_stores.warehouse_id_c42
WHERE synced_stores.warehouse_id_c42 IS NULL;


In [ ]:
New Users Line items

SELECT sales.* 
,case when customer_id is not Null and Channel ='Mobile' then 'Saathi App'
else Channel
end as channel2 from 
(SELECT
line_items.order_id as ord_id
,case when ols.channel = 1 then 'Mobile'
	when ols.channel = 0 then 'Call Center'
	when ols.channel = 4 then 'Website'
	else 'others' end as Channel
,case when orders.state =0 then 'cart'
when orders.state =1 then 'pre_checkout'
when orders.state =2 then 'Checkout'
when orders.state =3 then 'order_received'
when orders.state =4 then 'shipped'
when orders.state =5 then 'out_for_delivery'
when orders.state =6 then 'Delivered'
when orders.state =7 then 'Cancelled'
when orders.state =14  then 'Return_completed'
when orders.state =17 then 'rescheduled'
else 'In-Progress' end as detailed_status
,line_items.variant_id
,concat(line_items.order_id, line_items.variant_id) as order_variant_id
,ols.state_changed_on + time '5:30' as confirmed_on
,distribution_centers.dc_code
,products.name as product_name	
,categories.name as category
,orders.customer_id
,ols.modified_by_id
,orders.city_id
,orders.payment_method
,orders.auto_completed
,orders.order_total_paise*1.0/100 as order_value
,orders.shipping_total_paise*1.0/100 as Shipping_charge
,line_items.quantity	
,line_items.mrp_paise*1.0/100 as MRP	
,line_items.sales_price_paise*1.0/100 as Sales_price	
,line_items.total_paise*1.0/100 as Total_Price	
,line_items.discount_amount_paise*1.0/100 as Discount	
--,line_items.promotional_price_paise*1.0/100 as Promotional_price	
--,line_items.discount_percent	
--,line_items.updated_at	
--,variants.product_id	
--,products.sku	

from line_items 
left join variants on	
line_items.variant_id = variants.id	
left join  products	
on variants.product_id = products.id	
left join product_types on	
products.product_type_id = product_types.id	
left join categories	
on categories.id = product_types.category_id
left join 
order_status_logs as ols 
on ols.order_id = line_items.order_id
left join
orders on orders.id = line_items.order_id
left join distribution_centers
on orders.store_id = distribution_centers.id
where line_items.order_id in
(
--New user orders
SELECT ord_id FROM(Select T1.*,
SUM(order_logs) over (PARTITION BY user_id) as firts_order FROM
(Select user_tbl.*,tbl2.order_id,tbl2.confirmed_on,
row_number() over(PARTITION BY user_id order by tbl2.confirmed_on ASC) as order_logs
FROM (Select * from
(select orders.id as ord_id,user_id,created_by_id,orders.state, city_id,order_total_paise*1.0/100 as Order_Value
FROM orders where created_at >'2023-03-31 18:30:00' and state > 2 and created_by_id in
---Date Filter
(select id from users where created_at > '2023-03-31 18:30:00'))as t2 
left join
(select id,registration_source,created_at + time '5:30' as registration_date From users where created_at >'2023-03-31') as tbl
on t2.user_id = tbl.id
WHERE city_id in (13) and registration_source in (0,1,4)) as user_tbl
left join
(select order_id,state_changed_on + time '5:30' as confirmed_on,channel,to_state from order_status_logs where to_state = 3
AND ("order_status_logs"."state_changed_on" >= '2023-03-31 18:30:00.000000'
AND "order_status_logs"."state_changed_on" < now())) tbl2
on user_tbl.ord_id = tbl2.order_id) as T1) new_user left Join (Select user_id,number from phones) as phon
on phon.user_id = new_user.user_id
where order_logs = 1 and confirmed_on>'2023-12-31 18:30:00' 
--Orders New User ends
)
and ols.state_changed_on >= '2023-12-31 18:30:00.000000' and ols.to_state = 3) as sales

### Usage of specific promotion or coupon code in specific date or date range Coupon code Wise report

In [ ]:
--# Usage of specific promotion or coupon code in specific date or date range Coupon code Wise report

SELECT 
orders.id as order_id,promotions.name as promo_name,
--promotions.description,
promotions.id as promotion_id,
coupon_code,
starts_at, expires_at,
--cash_back_total_paise/100 as cashback
state_changed_on + time '5:30' as order_date,
case when channel = 0 then 'Call Center'
when channel = 1 then 'Mobile'
else  'Website' end as channel,
orders.user_id,
case 
when orders.state =0 then 'cart'
when orders.state =1 then 'pre_checkout'
when orders.state =2 then 'Checkout'
when orders.state =3 then 'order_received'
when orders.state =4 then 'shipped'
when orders.state =5 then 'out_for_delivery'
when orders.state =6 then 'Delivered'
when orders.state =7 then 'Cancelled'
when orders.state =14  then 'Return_completed'
when orders.state =17 then 'rescheduled'
else 'In-Progress' end as status, 
order_total_paise/100*1.0 as order_value
FROM order_promotions
inner join promotions on promotions.id = order_promotions.promotion_id
left join order_status_logs on order_status_logs.order_id = order_promotions.order_id
inner join orders on orders.id = order_promotions.order_id
where --coupon_code = 'FIRST100' and
order_promotions.promotion_id = 2137
and to_state = 3
and order_status_logs.state_changed_on > '2024-06-30 18:30' 
and order_status_logs.state_changed_on < '2024-07-31 18:30'
and orders.city_id = 13
order by state_changed_on desc

In [ ]:
SELECT 
op.order_id
,o.state
,osl.state_changed_on + time '5:30' as confirmed_on 
FROM order_promotions op
inner join orders o on o.id = op.order_id 
inner join order_status_logs osl on osl.order_id = op.order_id
where promotion_id = 2137
and to_state = 3
and osl.state_changed_on > '2024-06-30 18:30' 
and osl.state_changed_on < '2024-07-31 18:30'

In [ ]:
# Usage of promotion total date wise 

SELECT * FROM(SELECT 
op.order_id
,case 
when o.state =0 then 'cart'
when o.state =1 then 'pre_checkout'
when o.state =2 then 'Checkout'
when o.state =3 then 'order_received'
when o.state =4 then 'shipped'
when o.state =5 then 'out_for_delivery'
when o.state =6 then 'Delivered'
when o.state =7 then 'Cancelled'
when o.state =14  then 'Return_completed'
when o.state =17 then 'rescheduled'
else 'In-Progress' end as status
FROM order_promotions op
inner join orders o on o.id = op.order_id 
where promotion_id = 2137) as promo_orders
left join
(SELECT order_id,
osl.state_changed_on + time '5:30' as confirmed_on 
from order_status_logs osl
where
osl.state_changed_on > '2024-06-30 18:30' 
and osl.state_changed_on < '2024-07-31 18:30'
and osl.to_state = 3) osl_orders
on promo_orders.order_id = osl_orders.order_id 

In [ ]:
# Disease Profile

--- Code to check top disease profiles 
SELECT property_value,count(user_id) from orders
inner join line_items on line_items.order_id = orders.id
inner join item_properties on item_properties.item_id = line_items.variant_id 
where property_id in(1351) --and property_value = 'Hypercholesterolemia'
and city_id = 13
group by property_value
order by count(user_id)desc

-- Code to Fetch user based on different Disease pofile
SELECT distinct orders.user_id, property_value as disease_profile,name,email,number
from orders
inner join line_items on line_items.order_id = orders.id
inner join item_properties on item_properties.item_id = line_items.variant_id 
inner join users on users.id = orders.user_id
inner join phones on phones.user_id = orders.user_id
where property_id in (1351) and property_value in('Cardiovascular disease','Diabetic','Gastro Intestinal Disorder')


### List of chronic customers


In [ ]:
SELECT  distinct orders.user_id, property_value
from line_items
left join orders on line_items.order_id = orders.id
inner join item_properties on item_properties.item_id = line_items.variant_id 
inner join users on users.id = orders.user_id
inner join phones on phones.user_id = orders.user_id
where property_id in (1352) and property_value = 'Chronic'

### Order Summary 

In [ ]:
SELECT agg_channel,
	SUM(CASE WHEN Status = 'Cancelled' then order_value else 0 END)   as Cancel_value,
	SUM(CASE WHEN Status = 'Cancelled' then ord_count else 0 END)     as cancel_count,
	SUM(CASE WHEN Status = 'Delivered' then order_value else 0 END)   as Delivered_value,
	SUM(CASE WHEN Status = 'Delivered' then ord_count else 0 END)     as Delivered_count,
	SUM(CASE WHEN Status = 'RTO' then order_value else 0 END)         as RTO_value,
	SUM(CASE WHEN Status = 'RTO' then ord_count else 0 END)           as RTO_count,
	SUM(CASE WHEN Status = 'in_progress' then order_value else 0 END) as in_progress_value,
	SUM(CASE WHEN Status = 'in_progress' then ord_count else 0 END)   as in_progress_count,
	SUM(order_value) as order_val,
	Sum(ord_count) as order_count,
	SUM(CASE WHEN Status = 'Delivered' then ord_count else 0 END)/Sum(ord_count) as Delivery_Percent,
	SUM(CASE WHEN Status = 'Cancelled' then ord_count else 0 END)/Sum(ord_count) as Cancel_percent,
	SUM(CASE WHEN Status = 'in_progress' then ord_count else 0 END)/Sum(ord_count) as Cancel_percent
FROM
(SELECT CASE 
        WHEN state = 6 THEN 'Delivered'
        WHEN state = 7 THEN 'Cancelled'
        WHEN state = 14 THEN 'RTO'
        ELSE 'in_progress' 
    	END AS Status,
	case when channel = 0 and coalesce(delivery_remarks,'') not ilike '%frsaathi%'then 'Call Center' 
		 when channel = 4 then 'HealthApp Website'
		 when channel = 1 and coalesce(u.is_saathi,false) = false then 'HealthApp Mobile'
		 when channel = 1 and coalesce(u.is_saathi,false) = true then 'Saathi App'
		 when channel = 0 and coalesce(delivery_remarks,'') ilike '%frsaathi%' then 'Saathi CC'
	else 'Other' end agg_channel,
    SUM(orders.order_total_paise / 100 * 1.0) AS order_value,
    COUNT(orders.id) AS ord_count
FROM orders
	inner join order_status_logs ols on orders.id = ols.order_id and to_state=3 
	inner join users u on orders.user_id = u.id
	where state >= 3 	
		and ols.state_changed_on between '2024-04-30 18:30' and  '2024-05-27 18:30'
		and orders.city_id = 13
GROUP BY Status,agg_channel) subqery
group by agg_channel



# Wallet Credit and debit report

In [ ]:
Wallet Credit and debit report
SELECT
	
	wallets.user_id,
	transactions.health_wallet_id,
	transactions.total_credit_points,
	transactions.total_debit_points,
	wallets.earned_points,
 	wallets.spent_points,
 	wallets.available_points,
 	--wallets.created_at,
 	wallets.updated_at,
	transactions.credit_transaction_summary,
	transactions.debit_transaction_summary
	from
(SELECT
 	
    credit.health_wallet_id,
    COALESCE(SUM(credit.points), 0) AS total_credit_points,
    COALESCE(SUM(debit.points), 0) AS total_debit_points,
 	
    STRING_AGG(credit.summary, ', ') AS credit_transaction_summary,
    STRING_AGG(debit.summary, ', ') AS debit_transaction_summary
FROM 
    (SELECT 
         health_wallet_id, 
         points,
         summary
     FROM 
         public.health_wallet_transactions 
     WHERE 
         created_at > '2023-12-31 18:30:00' 
         AND transaction_type = 'credit') AS credit
LEFT JOIN 
    (SELECT 
         health_wallet_id, 
         points,
         summary
     FROM 
         public.health_wallet_transactions 
     WHERE 
         created_at > '2023-12-31 18:30:00' 
         AND transaction_type = 'debit') AS debit
ON credit.health_wallet_id = debit.health_wallet_id
GROUP BY credit.health_wallet_id) as transactions
left join 
(
SELECT
	* from health_wallets where updated_at > '2023-12-31 18:30:00' 
)as  wallets
on wallets.id = transactions.health_wallet_id
	



# Sales current month order

In [ ]:
SELECT
    line_items.order_id as ord_id,	
    line_items.variant_id,
    products.name as product_name,	
    categories.name as category,
    line_items.quantity,	
    line_items.mrp_paise * 1.0 / 100 as MRP,	
    line_items.sales_price_paise * 1.0 / 100 as Sales_price,	
    line_items.total_paise * 1.0 / 100 as Total_Price,	
    line_items.discount_amount_paise * 1.0 / 100 as Discount,
    CASE 
        WHEN osl.channel = 0 THEN 'Call Center'
        WHEN osl.channel = 1 THEN 'Mobile'
        ELSE 'Website' 
    END as channel_1,
    CASE 
        WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
        WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
        WHEN osl.channel = 0 THEN 'Call Center'
        WHEN osl.channel = 1 THEN 'Saathi_App'
        ELSE 'Website'
    END as channel,
    osl.state_changed_on + interval '5:30' hour as confirmed_on,
    CASE 
        WHEN o.state = 0 THEN 'cart'
        WHEN o.state = 1 THEN 'pre_checkout'
        WHEN o.state = 2 THEN 'Checkout'
        WHEN o.state = 6 THEN 'Delivered'
        WHEN o.state = 7 THEN 'Cancelled'
        WHEN o.state = 14 THEN 'Return_completed'
        ELSE 'In-Progress' 
    END as status,
    dc.dc_code,
	ip.property_value as d_profile_tbl,
	ip2.property_value as act_chronic
FROM 
    line_items 
    LEFT JOIN variants ON line_items.variant_id = variants.id	
    LEFT JOIN products ON variants.product_id = products.id	
    LEFT JOIN product_types ON products.product_type_id = product_types.id	
    LEFT JOIN categories ON categories.id = product_types.category_id
    LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
    LEFT JOIN orders o ON o.id = osl.order_id
    LEFT JOIN distribution_centers dc ON dc.id = o.store_id
	LEFT JOIN item_properties ip ON ip.item_id = line_items.variant_id AND ip.property_id IN ('1351')
	LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
WHERE 
    line_items.order_id IN (
        SELECT order_id 
        FROM order_status_logs as ols
        LEFT JOIN orders ON orders.id = ols.order_id
        WHERE 
            state_changed_on >= '2024-07-31 18:30:00.000000' 
            AND to_state = 3
            AND city_id = 13
    )
    AND osl.to_state = 3

# Saving File in a DF from DATABASE

In [ ]:
#C42 Current Month Orders without Duplicates -- Date Change Required
import csv
import psycopg2
import pandas as pd
import numpy as np
import paramiko
from sshtunnel import SSHTunnelForwarder
# EC2 instance connection details
ec2_host = '65.1.183.184'
ec2_username = 'ubuntu'
ec2_pem_key_path = 'C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem'
# AWS RDS PostgreSQL database connection details
rds_host = 'emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com'
rds_port = 5432
rds_database = 'efrprod'
rds_user = 'emamireaduser'
rds_password = 'emamireadaccess'
try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )
        cursor = connection.cursor()
        # Perform database operations here
        cursor.execute(
'''
SELECT * FROM areas
''')
        records = cursor.fetchall()
        # Create a DataFrame from the fetched records
        column_names = [desc[0] for desc in cursor.description] #This will fetch column name from DATABASE table 
        df = pd.DataFrame(records, columns=column_names)
except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

# AD Hoc Report Codes

In [ ]:
All Wallet Transaction of a user

SELECT * from health_wallet_transactions where health_wallet_id in
(select id from health_wallets where user_id in (543772))

# Ecogreen mapping

In [1]:
# Ecogreen mapping
import csv
import psycopg2
import pandas as pd
import numpy as np
import paramiko
from sshtunnel import SSHTunnelForwarder
# EC2 instance connection details
ec2_host = '65.1.183.184'
ec2_username = 'ubuntu'
ec2_pem_key_path = "C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem"
# AWS RDS PostgreSQL database connection details
rds_host = 'emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com'
rds_port = 5432
rds_database = 'efrprod'
rds_user = 'emamireaduser'
rds_password = 'emamireadaccess'
try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )
        cursor = connection.cursor()
#1190-qty per pack, 1184-ecogreen fr code mapping
        # Perform database operations here
        cursor.execute(
'''
SELECT * FROM health_wallets where available_points >0
''')
        records = cursor.fetchall()
        # Write records to a CSV file
        with open('wallet.csv', 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([desc[0] for desc in cursor.description])  # Write column headers
            writer.writerows(records)
        print("Data fetched and saved to itemcode_mapping.csv")
except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
print("Updated at", pd.Timestamp.now())

SyntaxError: invalid syntax (4055399771.py, line 1)

# shipped qty based on order variant id

In [ ]:
shipped qty based on order variant id

SELECT 
--invoices.order_id
concat(invoices.order_id, invoice_items.variant_id) as order_variant_id
--,invoice_items.invoice_id
--,invoice_items.variant_id
,SUM(invoice_items.shipped_qty) as shipped_qty_agg
--,invoices.invoiced_at
--,invoices.wallet_amount
--,invoices.number
FROM invoice_items 
LEFT JOIN invoices ON invoices.id = invoice_items.invoice_id
WHERE invoice_items.created_at > '2024-01-31 18:30:00' 
AND invoices.created_at > '2024-01-31 18:30:00'
group by order_variant_id




SELECT 
invoices.order_id
,invoice_items.variant_id
,SUM(invoice_items.quantity) as qty
,SUM(invoice_items.shipped_qty) as shipped_qty
FROM invoice_items 
left join 
invoices
on invoices.id = invoice_items.invoice_id
where invoices.created_at > '2024-02-29 18:30'
group by order_id , variant_id





# Ecom Mapping

In [ ]:
To download variants with ecogreen code


SELECT * FROM public.item_properties where property_id = 1184  #ecogreen fr code mapping

SELECT * FROM public.item_properties where property_id = 1190   #Qty per pack

In [ ]:
501,502 Warehouse inventory dump

SELECT
concat ( 'FR-',variant_id) as fr_code,
CASE
	WHEN warehouse_id = 12 then '501'
	else '502' END as Warehouse,
available_quantity,
property_value as eg_code,
last_synced_at
FROM public.warehouse_variants 
left join item_properties 
on item_properties.item_id = warehouse_variants.variant_id
and property_id = 1184
where warehouse_variants.warehouse_id in (2,12)


In [ ]:
SELECT * FROM health_wallet_transactions where health_wallet_id in(
SELECT health_wallet_id FROM public.health_wallet_transactions where 
    summary like '%New year gift%' or summary like '%Special Offer Gift%'
) and transaction_type = 'debit'

# To Fetch Database table details 

In [ ]:
#To Fetch Database table details 

        # Perform database operations here
        cursor.execute("""
            SELECT table_name, column_name
            FROM information_schema.columns 
            WHERE table_schema = 'public'
            ORDER BY table_name, ordinal_position
        """)
        table_columns = cursor.fetchall()

        # Convert the fetched data into a DataFrame
        df = pd.DataFrame(table_columns, columns=['Table Name', 'Column Name'])

        print("DataFrame of Tables and their Column Names:")
        df.to_csv('tables.csv')

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)


In [ ]:
To Get Concatenated column

SELECT concat(order_id,'',invoices.id ) AS order_variant_id
FROM invoices where created_at>'2024-01-31 18:30'


In [ ]:
#Saathi's wallet Balance
SELECT * FROM public.health_wallets where user_id in (SELECT users.id from users where users.roles like '%saathi%')


#To Fetch saathi where wallet balance is greater than 0
SELECT * FROM public.health_wallets where user_id in (SELECT users.id from users where users.roles like '%saathi%') 
and available_points > 0

In [ ]:
# Marketing User Journey Funnel

SELECT count(distinct user_id) as new_users 
FROM PHONES 
WHERE otp_requested_at > '2024-02-29 18:30:00' 
AND user_id IN (
    SELECT id 
    FROM users 
    WHERE created_at > '2024-02-29 18:30:00' 
    AND registration_source = 1
);

SELECT count(distinct user_id) as registered 
FROM PHONES 
WHERE otp_verified_at > '2024-02-29 18:30:00' 
AND user_id IN (
    SELECT id 
    FROM users 
    WHERE created_at > '2024-02-29 18:30:00' 
    AND registration_source = 1
);

SELECT count(distinct user_id) as added_to_cart 
FROM orders 
WHERE user_id IN (
    SELECT id 
    FROM users 
    WHERE created_at > '2024-02-29 18:30:00' 
    AND registration_source = 1
);

SELECT count(distinct user_id) as order_placed 
FROM orders 
WHERE user_id IN (
    SELECT id 
    FROM users 
    WHERE created_at > '2024-02-29 18:30:00' 
    AND registration_source = 1
) 
AND orders.state > 2;


In [ ]:
Saathi list


SELECT users.id from users where users.roles like '%saathi%'

In [ ]:
##--Pincode Mapping Data

SELECT 
        areas.pincode,
        areas.city_id,
        area_warehouses.fulfillment_center_level AS F_level,
        areas.name,
        warehouses.id AS warehouse_id,
        warehouses.store_name,
        warehouses.eg_code,
        delivery_tat
    FROM 
        area_warehouses
        LEFT JOIN areas ON areas.id = area_warehouses.area_id
        LEFT JOIN warehouses ON warehouses.id = area_warehouses.warehouse_id
    WHERE 
        city_id IN ('13')

In [ ]:
##-Sync Failures

SELECT
    sync_services.type,
    sync_services.type_order_id,
    CASE 
        WHEN sync_services.status = 0 THEN 'pending'
        WHEN sync_services.status = 1 THEN 'success'
        WHEN sync_services.status = 2 THEN 'failure'
    END as Sync_status,
    sync_services.order_id,
    sync_services.type_of_api,
    sync_services.created_at,
    sync_services.updated_at,
    sync_services.error_message,
    sync_services.retry_counts
FROM
    public.sync_services
WHERE
    updated_at >= CURRENT_DATE - INTERVAL '1 day'
AND
    updated_at < CURRENT_DATE
AND
    status in ('0','2')
AND
    sync_services.type in ('Ecogreen','Vinculum','Shipsy')

In [ ]:
# New Users With Count Of Orders

WITH users_1 AS (
SELECT * FROM(Select T1.*,
SUM(order_logs) over (PARTITION BY user_id) as firts_order FROM
(Select user_tbl.*,tbl2.order_id,tbl2.confirmed_on,
row_number() over(PARTITION BY user_id order by tbl2.confirmed_on ASC) as order_logs
FROM (Select * from
(select orders.id as ord_id,user_id,created_by_id,orders.state, city_id,order_total_paise*1.0/100 as Order_Value
FROM orders where created_at >'2024-01-31 18:30:00' and state > 2 and created_by_id in
---Date Filter
(select id from users where created_at > '2024-01-31 18:30:00'))as t2 
left join
(select id,registration_source,created_at + time '5:30' as registration_date From users where created_at >'2024-01-31') as tbl
on t2.user_id = tbl.id
WHERE city_id in (13) and registration_source in (0,1,4)) as user_tbl
left join
(select order_id,state_changed_on + time '5:30' as confirmed_on,channel,to_state from order_status_logs where to_state = 3
AND ("order_status_logs"."state_changed_on" >= '2024-01-31 18:30:00.000000'
AND "order_status_logs"."state_changed_on" < now())) tbl2
on user_tbl.ord_id = tbl2.order_id) as T1) new_user left Join (Select user_id,number from phones) as phon
on phon.user_id = new_user.user_id)
SELECT users_1.*,MAX(order_logs) OVER (PARTITION BY created_by_id) AS order_count
FROM users_1;


In [12]:
# Connection to Linux Machine
import psycopg2
from sshtunnel import SSHTunnelForwarder
try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        ('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey='C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem', #pem File path
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22) # 22 in Windows, 5432 in Linux
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='127.0.0.1',
            port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        # Perform database operations here
        cursor.execute(
            '''
SELECT
    sync_services.type,
    sync_services.type_order_id,
    CASE 
        WHEN sync_services.status = 0 THEN 'pending'
        WHEN sync_services.status = 1 THEN 'success'
        WHEN sync_services.status = 2 THEN 'failure'
    END as Sync_status,
    sync_services.order_id,
    sync_services.type_of_api,
    sync_services.created_at,
    sync_services.updated_at,
    sync_services.error_message,
    sync_services.retry_counts
FROM
    public.sync_services
WHERE
    updated_at >= CURRENT_DATE - INTERVAL '1 day'
AND
    updated_at < CURRENT_DATE
AND
    status in ('0','2')
AND
    sync_services.type in ('Ecogreen','Vinculum','Shipsy')
'''
        )
        records = cursor.fetchall()
        # Print records
        for record in records:
            print(record)
except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:# Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


('Shipsy', None, 'failure', 988594, 'create', datetime.datetime(2024, 2, 11, 8, 39, 25, 240727), datetime.datetime(2024, 2, 11, 11, 30, 12, 383404), '{"status":"OK","data":[{"reason":"BOOKING_THIRD_PARTY_VALIDATION_ERROR","success":false,"message":"Unable to create booking for following accounts:\\n [{\\"courier_partner\\":\\"ECOMEXPRESS\\",\\"courier_account\\":\\"ECOMEXPRESS\\",\\"error\\":\\"CONSIGNEE_PINCODE_NOT_SERVICED\\"}]","reference_number":"988594","customer_reference_number":"","courier_account":"ECOMEXPRESS","courier_partner":"ECOMEXPRESS"}]}', 6)
PostgreSQL connection is closed


In [ ]:
# Connection to Linux Machine
import psycopg2
import csv
from sshtunnel import SSHTunnelForwarder
try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        ('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey='C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem', #pem File path
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22) # 22 in Windows, 5432 in Linux
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='127.0.0.1',
            port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        # Perform database operations here
        cursor.execute(
            '''
--SELECT item_id as Variant_id, proprty_value as quantity FROM item_properties WHERE property_id in ('1190')
SELECT * FROM Invoice_items WHERE created_at > '2024-01-31 18:30:00'
'''
        )
        records = cursor.fetchall()
        # Write records to a CSV file
        with open('invoice_items.csv', 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([desc[0] for desc in cursor.description])  # Write column headers
            writer.writerows(records)
        print("Data fetched and saved to qty_per_pack.csv")
except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

In [ ]:
with cte_bounce_orders as (
SELECT 
    o.order_id,
    orders.state,
    v.sku,
    v.name,
    o.deficit_quantity,
    w.store_name AS Level_1_store,
    ar.pincode,
    ols.state_changed_on + INTERVAL '5 hours 30 minutes' AS confirmed_on,
    CASE
        WHEN ols.channel = '0' THEN 'call center'
        WHEN ols.channel = '1' THEN 'mobile'
        ELSE 'website'
    END AS channel
FROM 
    order_deficit_quantity_logs o
LEFT JOIN 
    warehouses w ON o.warehouse_id = w.id
LEFT JOIN 
    areas ar ON o.area_id = ar.id
LEFT JOIN 
    variants v ON v.id = o.variant_id
LEFT JOIN 
    orders ON orders.id = o.order_id
LEFT JOIN 
    order_status_logs ols ON ols.order_id = o.order_id
WHERE 
    ols.to_state = 3 
    AND ols.state_changed_on > '2024-01-31 18:30:00'
    AND orders.state > 2
    AND ols.state_changed_on + INTERVAL '5 hours 30 minutes' > '2024-02-28 18:30:00')
select * From cte_bounce_orders where confirmed_on > current_timestamp - interval '12 hour'

In [ ]:
#To Fetch Database table details 


        # Perform database operations here
        cursor.execute("""
            SELECT table_name, column_name
            FROM information_schema.columns 
            WHERE table_schema = 'public'
            ORDER BY table_name, ordinal_position
        """)
        table_columns = cursor.fetchall()

        # Convert the fetched data into a DataFrame
        df = pd.DataFrame(table_columns, columns=['Table Name', 'Column Name'])

        print("DataFrame of Tables and their Column Names:")
        df.to_csv('tables.csv')

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)


In [ ]:
#To Fetch Database table details 

import pandas as pd

try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )
        cursor = connection.cursor()

        # Perform database operations here
        cursor.execute("""
            SELECT table_name, column_name
            FROM information_schema.columns 
            WHERE table_schema = 'public'
            ORDER BY table_name, ordinal_position
        """)
        table_columns = cursor.fetchall()

        # Organize the results into a dictionary where keys are table names and values are lists of column names
        tables = {}
        for table, column in table_columns:
            if table in tables:
                tables[table].append(column)
            else:
                tables[table] = [column]

        # Convert the dictionary to a DataFrame
        df = pd.DataFrame.from_dict(tables, orient='index', columns=['Column Names'])

        # Print the DataFrame
        print("DataFrame of Tables and their Column Names:")
        print(df)

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)

finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
